In [ ]:
import time

import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from util import display_image

In [ ]:
img = cv.imread('../samples/KakaoTalk_Photo_2020-02-02-12-26-11.jpeg')
# img = cv.imread('../img/paper.jpg')

display_image(img, right=2, top=2)

draw = img.copy()

n_x = 20
n_y = 5
kernel = np.ones(shape=[n_x, n_y]) / (n_x*n_y)

gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
# gray = cv.GaussianBlur(gray, (5, 5), 0)
# gray = cv.bilateralFilter(gray, 5, 75, 75)
gray = cv.filter2D(gray, -1, kernel)

ret, thres = cv.threshold(gray, 170, 255, cv.THRESH_BINARY)

display_image(thres)

k = cv.getStructuringElement(cv.MORPH_RECT, (300, 30))

erosion = cv.morphologyEx(thres, cv.MORPH_CLOSE, k)

display_image(erosion)





src = erosion.copy()

draw = img.copy()

cnts, _ = cv.findContours(src, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

# edged = cv.Canny(thres, 50, 200)

# display_image(edged, right=2, top=2)

cnts = sorted(cnts, key=cv.contourArea, reverse=True)

len(cnts)

for c in cnts:
    peri = cv.arcLength(c, True)    
    vertices = cv.approxPolyDP(c, 0.1*peri, True)
    
    if len(vertices) == 4:
        break

cnts[0].shape

vertices

pts = vertices.reshape(4, 2)

pts

for x, y in pts:
    cv.circle(draw, (x, y), 30, (0, 255, 0), -1)

display_image(draw, right=2, top=2)







merged = np.hstack([img, draw])

sm = pts.sum(axis=1)

diff = np.diff(pts, axis=1)

topLeft = pts[np.argmin(sm)]
bottomRight = pts[np.argmax(sm)]
topRight = pts[np.argmin(diff)]
bottomLeft = pts[np.argmax(diff)]

pts1 = np.float32([topLeft, topRight, bottomRight, bottomLeft])

w1 = abs(bottomRight[0] - bottomLeft[0])
w2 = abs(topRight[0] - topLeft[0])
h1 = abs(topRight[1] - bottomRight[1])
h2 = abs(topLeft[1] - bottomLeft[1])

width = max([w1, w2])
height = max([h1, h2])

pts2 = np.float32([[0, 0], [width-1, 0], [width-1, height-1], [0, height-1]])

mtrx = cv.getPerspectiveTransform(pts1, pts2)

result = cv.warpPerspective(img, mtrx, (width, height))

display_image(result, right=4, top=4)





